In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 25.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import csv
import mediapipe as mp
import matplotlib.pyplot as plt
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

In [ ]:
video_path = "/content/drive/MyDrive/ASL_Apple.mov"
cap = cv2.VideoCapture(video_path)

# Create a dictionary to store the landmark data
landmark_data_dict = {}

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    frame_number = 0
    process_counter = 0

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break
        if process_counter % 3 == 0:
          image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          results = holistic.process(image_rgb)

          #if results.face_landmarks and results.left_hand_landmarks and results.pose_landmarks and results.right_hand_landmarks:
              # Process face landmarks
          landmark_data_dict[frame_number] = {}
          face_landmarks = results.face_landmarks.landmark
          for idx, landmark in enumerate(face_landmarks):
                landmark_key = f"x_face_{idx}"
                landmark_data_dict[frame_number][landmark_key] = landmark.x
                landmark_key = f"y_face_{idx}"
                landmark_data_dict[frame_number][landmark_key] = landmark.y
                landmark_key = f"z_face_{idx}"
                landmark_data_dict[frame_number][landmark_key] = landmark.z

                landmark_key = f"x_left_hand_{idx}"
                landmark_data_dict[frame_number][landmark_key] = "NaN"
                landmark_key = f"y_left_hand_{idx}"
                landmark_data_dict[frame_number][landmark_key] = "NaN"
                landmark_key = f"z_left_hand_{idx}"
                landmark_data_dict[frame_number][landmark_key] = "NaN"

              # Process left hand landmarks
          #left_hand_landmarks = results.left_hand_landmarks.landmark
          #for idx, landmark in enumerate(left_hand_landmarks):
                #landmark_key = f"x_left_hand_{idx}"
                #landmark_data_dict[landmark_key] = landmark.x
                #landmark_key = f"y_left_hand_{idx}"
                #landmark_data_dict[landmark_key] = landmark.y
              # landmark_key = f"z_left_hand_{idx}"
              # landmark_data_dict[landmark_key] = landmark.z

            # Process pose landmarks
          pose_landmarks = results.pose_landmarks.landmark
          for idx, landmark in enumerate(pose_landmarks):
                landmark_key = f"x_pose_{idx}"
                landmark_data_dict[frame_number][landmark_key] = landmark.x
                landmark_key = f"y_pose_{idx}"
                landmark_data_dict[frame_number][landmark_key] = landmark.y
                landmark_key = f"z_pose_{idx}"
                landmark_data_dict[frame_number][landmark_key] = landmark.z

              # Process right hand landmarks
          if results.right_hand_landmarks:
            right_hand_landmarks = results.right_hand_landmarks.landmark
            for idx, landmark in enumerate(right_hand_landmarks):
                landmark_key = f"x_right_hand_{idx}"
                landmark_data_dict[frame_number][landmark_key] = landmark.x
                landmark_key = f"y_right_hand_{idx}"
                landmark_data_dict[frame_number][landmark_key] = landmark.y
                landmark_key = f"z_right_hand_{idx}"
                landmark_data_dict[frame_number][landmark_key] = landmark.z
          else:
            for idx in range(21):
                landmark_key = f"x_right_hand_{idx}"
                landmark_data_dict[frame_number][landmark_key] = "NaN"
                landmark_key = f"y_right_hand_{idx}"
                landmark_data_dict[frame_number][landmark_key] = "NaN"
                landmark_key = f"z_right_hand_{idx}"
                landmark_data_dict[frame_number][landmark_key] = "NaN"

          # Draw the landmarks on the frame (optional)
          annotated_image = frame.copy()
          mp_drawing.draw_landmarks(annotated_image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
          mp_drawing.draw_landmarks(annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
          mp_drawing.draw_landmarks(annotated_image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
          mp_drawing.draw_landmarks(annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

          plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
          plt.axis('off')
          plt.show()
          frame_number += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        process_counter += 1

    cap.release()
    cv2.destroyAllWindows()



Output hidden; open in https://colab.research.google.com to view.

In [ ]:
csv_file = "landmark_data.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header row
    header_row = ["frame"]
    for key in landmark_data_dict[1].keys():
        header_row.append(key)
    writer.writerow(header_row)

    # Write the data rows
    for frame_num, landmark_data in landmark_data_dict.items():
        data_row = [frame_num]
        for key, value in landmark_data.items():
            data_row.append(value)
        writer.writerow(data_row)

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('/content/landmark_data.csv')
df["sequence_id"] = 1
df.to_parquet('output.parquet')
df

,frame,x_face_0,y_face_0,z_face_0,x_left_hand_0,y_left_hand_0,z_left_hand_0,x_face_1,y_face_1,z_face_1,...,x_right_hand_18,y_right_hand_18,z_right_hand_18,x_right_hand_19,y_right_hand_19,z_right_hand_19,x_right_hand_20,y_right_hand_20,z_right_hand_20,sequence_id
0,0,0.533667,0.288551,-0.026091,NaN,NaN,NaN,0.531371,0.258166,-0.045169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1,0.536458,0.288869,-0.025997,NaN,NaN,NaN,0.534018,0.258519,-0.045639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2,0.535968,0.288742,-0.026353,NaN,NaN,NaN,0.533252,0.258873,-0.045829,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,3,0.534478,0.288597,-0.026606,NaN,NaN,NaN,0.531515,0.258408,-0.045866,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,4,0.533058,0.289170,-0.026527,NaN,NaN,NaN,0.530190,0.258494,-0.045535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,126,0.571093,0.315286,-0.023301,NaN,NaN,NaN,0.563705,0.284939,-0.044306,...,0.238186,0.346175,-0.037162,0.246206,0.370100,-0.033593,0.241445,0.383443,-0.026178,1
127,127,0.570823,0.317128,-0.022505,NaN,NaN,NaN,0.563333,0.286958,-0.044386,...,0.237506,0.346387,-0.036606,0.245102,0.370365,-0.032973,0.240016,0.383025,-0.025575,1
128,128,0.570272,0.319428,-0.021880,NaN,NaN,NaN,0.562886,0.289337,-0.044728,...,0.237922,0.347803,-0.036604,0.244941,0.372013,-0.032996,0.239398,0.384612,-0.025680,1
129,129,0.569147,0.319798,-0.021587,NaN,NaN,NaN,0.561463,0.290341,-0.044911,...,0.237797,0.348814,-0.036062,0.244467,0.372982,-0.032502,0.238655,0.385273,-0.025178,1


In [ ]:
data = {
    "sequence_id": [1],
    "phrase": ["abcdef"]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv("new_data.csv", index=False)

NameError: ignored